In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import LlamaConfig, LlamaForCausalLM, AutoTokenizer

/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2023-09-18 08:09:23,077] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
!ls -lh fpf-13b-32k

total 8.0K
drwxr-xr-x 3 ubuntu ubuntu 4.0K Sep 18 06:45 checkpoint-6900
drwxr-xr-x 3 ubuntu ubuntu 4.0K Sep 18 07:55 checkpoint-6950


In [4]:
half_config = LlamaConfig.from_pretrained('fpf-13b-32k/checkpoint-6950', 
                                             num_hidden_layers = 8)
half_config

LlamaConfig {
  "_name_or_path": "mesolitica/llama-13b-hf-16384-fpf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 32768,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 8,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
small = LlamaForCausalLM(half_config)

In [ ]:
sum(p.numel() for p in small.parameters())

In [ ]:
small

In [ ]:
model = LlamaForCausalLM.from_pretrained('fpf-13b-32k/checkpoint-6950')

In [ ]:
import torch.nn as nn

def copy_layers(src_layers, dest_layers, layers_to_copy):
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

layers_to_copy = [0,1,2,3,4,5,6,7]

copy_layers(model.model.layers, small.model.layers, layers_to_copy)

In [ ]:
small.model.embed_tokens.load_state_dict(model.model.embed_tokens.state_dict())

In [ ]:
small.model.norm.load_state_dict(model.model.norm.state_dict())

In [ ]:
small.lm_head.load_state_dict(model.lm_head.state_dict())

In [ ]:
small.save_pretrained("3b")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('fpf-13b-32k/checkpoint-6950')
tokenizer.save_pretrained("3b")

In [ ]:
!ls -lh 3b